In [1]:
import sys
import matplotlib.pyplot as plt
%matplotlib inline



In [2]:
from autograd.numpy import pi, mat, cos, sin, zeros, eye, array
import autograd.numpy as np
from autograd.numpy import concatenate

def rotx(theta):
    """
    Rotation about X-axis

    @type theta: number
    @param theta: the rotation angle
    @rtype: 3x3 orthonormal matrix
    @return: rotation about X-axis

    @see: L{roty}, L{rotz}, L{rotvec}
    """

    ct = cos(theta)
    st = sin(theta)
    return np.array([[1,  0,    0,  0],
                     [0,  ct, -st,  0],
                     [0,  st,  ct,  0],
                     [0,  0,    0,  1]])

def roty(theta):
    """
    Rotation about Y-axis

    @type theta: number
    @param theta: the rotation angle
    @rtype: 3x3 orthonormal matrix
    @return: rotation about Y-axis

    @see: L{rotx}, L{rotz}, L{rotvec}
    """

    ct = cos(theta)
    st = sin(theta)

    return np.array([[ct,   0,   st, 0],
                     [0,    1,    0, 0],
                     [-st,  0,   ct, 0],
                     [  0,  0,    0, 1]])

def rotz(theta):
    """
    Rotation about Z-axis

    @type theta: number
    @param theta: the rotation angle
    @rtype: 3x3 orthonormal matrix
    @return: rotation about Z-axis

    @see: L{rotx}, L{roty}, L{rotvec}
    """

    ct = cos(theta)
    st = sin(theta)

    return np.array([[ct,      -st,  0, 0],
                     [st,       ct,  0, 0],
                     [ 0,        0,  1, 0],
                     [ 0,        0,  0, 1]])

def trotx(theta):
    return rotx(theta)

def troty(theta):
    return roty(theta)

def trotz(theta):
    return rotz(theta)


##################### Euler ang
def transl(x, y=None, z=None):
    return np.array([[1., 0, 0, x],
                     [0, 1, 0, y],
                     [0, 0, 1, z],
                     [0, 0, 0, 1]])

def r2t(R):
    """
    Convert a 3x3 orthonormal rotation matrix to a 4x4 homogeneous transformation::

        T = | R 0 |
            | 0 1 |

    @type R: 3x3 orthonormal rotation matrix
    @param R: the rotation matrix to convert
    @rtype: 4x4 homogeneous matrix
    @return: homogeneous equivalent
    """

    return concatenate( (concatenate( (R, zeros((3,1))),1), np.array([0,0,0,1])) )


Tz = lambda z: transl(x=0, y=0, z=z)
Ty = lambda y: transl(x=0, y=y, z=0)
Tx = lambda x: transl(x=x, y=0, z=0)
Rx = lambda x: trotx(x)
Ry = lambda y: troty(y)
Rz = lambda z: trotz(z)


In [3]:
L1Z = 100; L2X = -25; L2Z = 12; L3Z = 100; L4Z = 100; L5Z = 25; L5X = 50; L6X = 150;
def dodot(lst):
    current = lst[0]
    for res in lst[1:]:
        current = np.dot(current, res)
    return current

def robot_function_base(q1, q2, q3, q4):
    return dodot([Tz(L1Z),Rz(q1),Tx(L2X),Tz(L2Z),Ry(-q2),Tz(L3Z),Ry(-q3),Tz(L4Z),Ry(-q4),Tz(L5Z),Tx(L5X),Tx(L6X)])


In [4]:
from __future__ import absolute_import, division
from __future__ import print_function
import autograd.numpy as np
import autograd.numpy.random as npr
from autograd.scipy.misc import logsumexp
from autograd import grad
from autograd.util import flatten
from autograd.optimizers import adam



In [5]:
from autograd.optimizers import sgd

In [ ]:
np.zeros_like(target)

In [6]:
def init_random_params(scale, rs=npr.RandomState(0)):
    """Build a list of parameters, xt, yt, zt, alpha, beta, gamma"""
    return [scale * rs.randn() for i in range(6)]

def armpos_predict(params, inputs):
    """ predict the robots position as measured by camera"""
    xt, yt, zt, alpha, beta, gamma = params
    q1, q2, q3, q4 = inputs
    armpos_robot = robot_function_base(q1,q2,q3,q4)
    armpos_cam = dodot( [armpos_robot, transl(xt,yt,zt), rotz(alpha), roty(beta), rotx(gamma)] )  
    return armpos_cam[:3,3]/1000

def l2_norm(params):
    """Computes l2 norm of params by flattening them into a vector."""
    flattened, _ = flatten(params)
    return np.dot(flattened, flattened)

def mean_squared_error(params, inputs, targets):
    err = 0
    for (inp, target) in zip(inputs, targets):
        err+=l2_norm(target - armpos_predict(params, inp[:4]))
    return err / len(inputs)



In [13]:
inps, trgt = zip(*train)
mean_squared_error(optimized_params_3, inps,trgt)

0.074697074685437181

In [16]:

optimized_params_3

[-317.29848279750149,
 -25.527708083310113,
 269.33728695996098,
 0.4481786398402916,
 0.37351159802999351,
 -0.19545557597528221]

In [7]:
import cPickle
with open('/data/robotics/motors_to_camera/motor_to_measured.pkl') as f:
    data = cPickle.load(f)

In [9]:
train = data[:120]
test = data[120:]

In [10]:
batch_size = 4
param_scale = 0.2
init_params = [-292.06198881701295,
 -22.784240722069956,
 275.69960736375606,
 0.4481786398402916,
 0.37351159802999351,
 -0.19545557597528221]

num_batches = int(np.ceil(len(train) / batch_size))
def batch_indices(iter):
    idx = iter % num_batches
    return slice(idx * batch_size, (idx+1) * batch_size)

    # Define training objective
def objective(params, iter):
    idx = batch_indices(iter)
    pos, target = zip(*train[idx])
    return mean_squared_error(params, pos, target)

objective_grad = grad(objective)

print("     Epoch     |    Train accuracy  |       Test accuracy  ")
def print_perf(params, iter, gradient):
    if iter % 100000 == 0:
        idx = batch_indices(iter)
        pos, target = zip(*train[idx])
        mse = mean_squared_error(params, pos, target)
        print("{:15}|{:20}".format(iter, mse))

    # The optimizers provided can optimize lists, tuples, or dicts of parameters.
optimized_params_3 = adam(objective_grad, init_params, step_size=0.0001,
                            num_iters=400000, callback=print_perf)


     Epoch     |    Train accuracy  |       Test accuracy  
              0|     0.0246849378456
         100000|     0.0977723604134
         200000|     0.0300247001383
         300000|      0.026946267995


In [ ]:
errs = np.zeros((200,), dtype=np.float)

for i in range(200):
    inp_ts, targ_ts = train[i]
    errs[i] = l2_norm(armpos_predict(optimized_params_2, inp_ts[:4])- targ_ts)

In [11]:
optimized_params_3

[-317.29848279750149,
 -25.527708083310113,
 269.33728695996098,
 0.4481786398402916,
 0.37351159802999351,
 -0.19545557597528221]

In [ ]:
new_train = [train[i] for i in range(200) if errs[i] < 0.25]

In [ ]:
plt.hist(errs)

0.024666666666666667

### Using sympy to check homogenous transformation matrix

In [ ]:
import sympy
xt, yt, zt, alpha, beta, gamma = sympy.symbols("xt, yt, zt, alpha, beta, gamma")

def sym_rotx(theta):
    ct = sympy.cos(theta)
    st = sympy.sin(theta)
    return sympy.Matrix([[1,  0,    0,  0],
                     [0,  ct, -st,  0],
                     [0,  st,  ct,  0],
                     [0,  0,    0,  1]])

def sym_roty(theta):
    ct = sympy.cos(theta)
    st = sympy.sin(theta)

    return sympy.Matrix([[ct,   0,   st, 0],
                     [0,    1,    0, 0],
                     [-st,  0,   ct, 0],
                     [  0,  0,    0, 1]])

def sym_rotz(theta):
    ct = sympy.cos(theta)
    st = sympy.sin(theta)

    return sympy.Matrix([[ct,      -st,  0, 0],
                     [st,       ct,  0, 0],
                     [ 0,        0,  1, 0],
                     [ 0,        0,  0, 1]])

def transl(x, y=None, z=None):
    return sympy.Matrix([[1., 0, 0, x],
                     [0, 1, 0, y],
                     [0, 0, 1, z],
                     [0, 0, 0, 1]])



transl(xt,yt,zt) * sym_rotz(alpha)*sym_roty(beta)*sym_rotx(gamma)

##  Learning motor errors and the homogenous transformation matrix

In [23]:
def c(q, a, b):
    return q*a + b

def armpos_predict(params, inputs):
    """ predict the robots position as measured by camera"""
    xt, yt, zt, alpha, beta, gamma, a1, a2, a3, a4, b1, b2, b3, b4 = params
    q1, q2, q3, q4 = inputs
    armpos_robot = robot_function_base(c(q1, a1, b1), c(q2, a2, b2), c(q3, a3, b3), c(q4, a4, b4))
    armpos_cam = dodot( [armpos_robot, transl(xt,yt,zt), rotz(alpha), roty(beta), rotx(gamma)] )  
    return armpos_cam[:3,3]/1000



In [24]:
batch_size = 4
param_scale = 0.2
init_params = [-317.29848279750149,
 -25.527708083310113,
 269.33728695996098,
 0.4481786398402916,
 0.37351159802999351,
 -0.19545557597528221] + [1]*4 + [0]*4

num_batches = int(np.ceil(len(train) / batch_size))
def batch_indices(iter):
    idx = iter % num_batches
    return slice(idx * batch_size, (idx+1) * batch_size)

    # Define training objective
def objective(params, iter):
    idx = batch_indices(iter)
    pos, target = zip(*train[idx])
    return mean_squared_error(params, pos, target)

objective_grad = grad(objective)

print("     Epoch     |    Train accuracy  |       Test accuracy  ")
def print_perf(params, iter, gradient):
    if iter % 100000 == 0:
        idx = batch_indices(iter)
        pos, target = zip(*train[idx])
        mse = mean_squared_error(params, pos, target)
        print("{:15}|{:20}".format(iter, mse))

    # The optimizers provided can optimize lists, tuples, or dicts of parameters.
optimized_params_4 = adam(objective_grad, init_params, step_size=0.001,
                            num_iters=400000, callback=print_perf)


     Epoch     |    Train accuracy  |       Test accuracy  
              0|     0.0277547965417
         100000|     0.0118730866665
         200000|    0.00458435147025
         300000|    0.00339910908735


### optimized_params_4

In [30]:
inps, trgt = zip(*test)
mean_squared_error(optimized_params_4, inps,trgt)

0.013762171556793228

In [32]:
np.sqrt(0.013762171556793228)

0.11731228220776045